In [ ]:
### This was a self-driven project I worked on to recieve extra credit for an undergraduate course in data analysis (CS 3501).
### I was given an open-ended computer vision problem - building a machine learning algorithim which could take in pictures of
### handwritten numbers, and determine what number the picture represented. To tackle this, I implemented a K-Means Clustering
### approach using Python 

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import time
import random
path = "/Users/willsayre/EC/"
xTrain = pd.read_csv(path + "trainX.txt", header = None)
xTest = pd.read_csv(path + "testX.txt", header = None)
yTrain = pd.read_csv(path + "trainY.txt", header = None)
yTest = pd.read_csv(path + "testY.txt", header = None)

In [6]:
##3

##Takes in a df of testing data, and trained clusters, and returns classification for each row of testing data
def oneNN(testX, clusters):
    labels = clusters['Labels']
    clusters.drop('Labels', axis=1, inplace=True)
    outputV = np.empty(len(testX))
    counter = 0
    for indexX, rowX in testX.iterrows():
        minDist = 0
        first = True 
        currentLab = 0
        for indexC, rowC in clusters.iterrows():
            dist = np.linalg.norm(rowX - rowC)
            if (first == True):
                minDist = dist
                currentLab = labels[indexC]
                first = False
            elif (dist < minDist):
                minDist = dist 
                currentLab = labels[indexC]
        outputV[counter] = currentLab
        counter = counter + 1
    return outputV


In [7]:
#Takes in data from a single class, and returns k clusters representing that class
def clusterInit(df, k):
    clusters = pd.DataFrame()
    ##Generate Random initailizations
    for kv in range (k):
        r = random.randint(0, len(df) - 1)
        centroid = df.iloc[r]
        clusters[kv] = centroid
    clusters = clusters.T
    closestLabs = np.empty(len(df))
    
    ##Iterate through 20 times
    for i in range (20):
        counter = 0
        for indexD, rowD in df.iterrows():
            minDist = 0
            first = True
            currentLab = 0
            
            ##Finds nearest centroid to each row of training data
            for indexC, rowC in clusters.iterrows():
                dist = np.linalg.norm(rowD - rowC)
                if (first == True):
                    minDist = dist
                    currentLab = indexC
                    first = False
                elif (dist < minDist):
                    minDist = dist
                    currentLab = indexC
            closestLabs[counter] = currentLab
            counter = counter + 1
        
        #Sets each cluster's value to mean of each member
        for kv in range(k):
            newCentroid = pd.DataFrame()
            for l in range(len(closestLabs)):
                if (closestLabs[l] == kv):
                    newCentroid[l] = df.iloc[l]
            newCentroid = newCentroid.T
            newCentroid = newCentroid.mean(axis = 0)
            clusters.iloc[kv] = newCentroid
            
    return clusters
                                           
                
                    
                
        
        

In [8]:
####Implementation
##Tests out K = 1 to K = 10 to determine optimal number of clusters for most accuracte classification. 
for k in range (1, 11):
    startTime = time.perf_counter()
    
    nndf = pd.DataFrame()
    for i in range (0, 10):
        df = pd.DataFrame() 
        counter = 0
        for index, row in yTrain.iterrows():
            if (row[0] == i):
                newRow = xTrain.iloc[index]
                df[index] = newRow
        df = df.T

        cluster = clusterInit(df, k)
        clusterdf = pd.DataFrame(cluster)
        labels = np.full((k),i)
        clusterdf["Labels"] = labels
        nndf = nndf.append(clusterdf, ignore_index = True)
    outputV = oneNN(xTest, nndf)
    resultsV = np.empty(len(outputV))
    counter = 0
    for index, row in yTest.iterrows():
        if (row[0] == outputV[counter]):
            resultsV[counter] = 1
        else:
            resultsV[counter] = 0
        counter = counter + 1
    accuracy = np.mean(resultsV)
    
    
    
    
    
    
    endTime = time.perf_counter()
    timeElapsed = endTime - startTime
    print("K = " + str(k))
    print("Accuracy = " + str(accuracy))
    print("Test Time = " + str(timeElapsed))
    print("\n")



K = 1
Accuracy = 0.7844990548204159
Test Time = 12.105646185000012


K = 2
Accuracy = 0.8147448015122873
Test Time = 18.28506793999999


K = 3
Accuracy = 0.8355387523629489
Test Time = 23.425464369999986


K = 4
Accuracy = 0.8449905482041588
Test Time = 30.369938306000023


K = 5
Accuracy = 0.831758034026465
Test Time = 35.034147622999996


K = 6
Accuracy = 0.8582230623818525
Test Time = 39.80371018400004


K = 7
Accuracy = 0.8506616257088847
Test Time = 48.28152271699997


K = 8
Accuracy = 0.8638941398865785
Test Time = 51.73392181599996


K = 9
Accuracy = 0.8638941398865785
Test Time = 58.150564282000005


K = 10
Accuracy = 0.8601134215500945
Test Time = 64.28727341900003




In [743]:
###Test time appears to be increaseing relatively linearly. This linear increase in time suggests that the K-means algorithim is O(n). 
##While test time increases monotonically, accuracy does not appear to follow this same pattern. There is a general increase in accuracy 
#as K increases from K=0 to K = 10, although this correlation does not always appear to be positive. Accuracy begins to plateau from K = 4 to K = 10.
##This suggests that adding clusters does not always help improve accuracy. I would imagine that, at a certain point, increasing K leads to overfitting.


##I would choose 9 clusters for this dataset since this had the highest accuracy. With an accuracy score of 0.86, this model proved highly effective at classifying
## handwritten numbers. The next step would be finding additional data to train the model with